## writing NaiveBayes Classifier

In [1]:
import numpy as np

In [2]:
def fit(x_train,y_train):
    result={}
    class_values=set(y_train)
    for current_class in class_values:
        result[current_class]={}
        result['total_data']=len(y_train)
        current_class_rows=(y_train==current_class)
        x_train_current=x_train[current_class_rows]
        y_train_current=y_train[current_class_rows]
        num_features=x_train.shape[1]
        result[current_class]['total_count']=len(y_train_current)
        for j in range(1,1+num_features):
            result[current_class][j]={}
            all_possible_values=set(x_train[:,j-1])
            for current_value in all_possible_values:
                result[current_class][j][current_value]=(x_train_current[:,j-1]==current_value).sum()
                
                
    
    return result

In [3]:
# we are taking log cozz multiplying prob is tending to zero....  a*b log(a)+log(b)
def probability(dictt,x,current_class):
    output=np.log(dictt[current_class]['total_count'])-np.log(dictt['total_data'])     # *prob(y=ai)
    num_features=len(dictt[current_class].keys()) - 1          # -1 for total count
    for j in range(1,num_features+1):
        xj=x[j-1]
        count_current_class_with_value_xj=dictt[current_class][j][xj] + 1
        count_current_class=dictt[current_class]['total_count'] + len(dictt[current_class][j].keys())
        current_xj_prob=np.log(count_current_class_with_value_xj)-np.log(count_current_class)
        output=output+current_xj_prob
    return output

In [4]:
def predictSinglePoint(dictt,x):
    classes =dictt.keys()
    best_p=-1000
    best_class=-1
    first_run=True
    for current_class in classes:
        if(current_class=='total_data'):
            continue
        p_current_class=probability(dictt,x,current_class)
        if(first_run or p_current_class>best_p):
            best_p=p_current_class
            best_class=current_class
            first_run=False
    
    return best_class

In [5]:
def predict(dictt,x_test):
    y_pred=[]
    for x in x_test:
        x_class=predictSinglePoint(dictt,x)
        y_pred.append(x_class)
    return(y_pred)

## converting dataset

In [6]:
def makelabelled(column):
    second_lmt=column.mean()
    first_lmt=0.5*second_lmt
    third_lmt=1.5*second_lmt
    for i in range(0,len(column)):
        if(column[i]<first_lmt):
            column[i]=0
        elif(column[i]<second_lmt):
            column[i]=1
        elif(column[i]<third_lmt):
            column[i]=2
        else:
            column[i]=3
    return(column)

In [7]:
from sklearn import datasets
iris=datasets.load_iris()
x=iris.data
y=iris.target

In [8]:
for i in range(0,x.shape[-1]):
    x[:,-1]=makelabelled(x[:,i])

In [9]:
from sklearn import model_selection
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.25,random_state=0)

In [10]:
dictt=fit(x_train,y_train)

In [11]:
y_pred=predict(dictt,x_test)

In [12]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.64      0.88      0.74        16
           2       0.60      0.33      0.43         9

    accuracy                           0.74        38
   macro avg       0.75      0.68      0.69        38
weighted avg       0.75      0.74      0.73        38

[[11  2  0]
 [ 0 14  2]
 [ 0  6  3]]
